# Leveraging SQL for Business Decisions of a Record Store

### The Database
SQL remains one of the most important tools in connecting with databases and answering business questions. In this project, I use SQL to connect to the [Chinook database](https://github.com/lerocha/chinook-database), a sample database available for MySQL, SQL Server, Oracle, etc.

The Chinook data model represents a digital media store which includes tables for artists, albums, media tracks, invoices and customers.

### The Goal

I intend to write SQL queries and subqueries to analyze the data found in the Chinook database to answer some hypothetical business questions.

In [1]:
# connecting Jupyter to the database file

%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Quick Query for Data Exploration

I'll start by getting familiar with the data. I can query the database to get a list of all tables and views in the database:

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


# Selecting Albums to Offer in Store

### The Problem

The Chinook record store has just signed a deal with a new record label, and I've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - I have the artist names, and the genre of music they produce:

#### Artist Name	| Genre
------------------
- Regal | Hip-Hop
- Red Tone | Punk
- Meteor and the Girls | Pop
- Slim Jim Bites | Blues

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so I'm interested in finding out which genres sell the best in the USA.

**I need to write a query to find out which genres sell the most tracks in the USA and make a recommendation for the three artists whose albums we should purchase for the store.**

In [3]:
%%sql

WITH
    usa_tracks AS
        (SELECT
            *
            FROM customer AS c
            WHERE country = 'USA'),
    tracks_sold_by_genre AS
        (SELECT
            usa.country AS country,
            g.genre_id AS genre_id,
            g.name AS genre,
            COUNT(il.invoice_line_id) AS tracks_sold
        FROM usa_tracks AS usa
            LEFT JOIN invoice AS i
                ON i.customer_id = usa.customer_id
            LEFT JOIN invoice_line AS il
                ON il.invoice_id = i.invoice_id
            LEFT JOIN track AS t
                ON t.track_id = il.track_id
            LEFT JOIN genre AS g
                ON g.genre_id = t.genre_id
        GROUP BY genre)

SELECT
    tsg.country,
    tsg.genre_id,
    tsg.genre,
    tsg.tracks_sold,
    ROUND(
            CAST(tsg.tracks_sold AS FLOAT) 
            / 
            (SELECT COUNT(*) FROM usa_tracks)
                                                , 3)
        AS percentage_sold
FROM tracks_sold_by_genre AS tsg

GROUP BY tsg.genre
ORDER BY percentage_sold DESC

Done.


country,genre_id,genre,tracks_sold,percentage_sold
USA,1,Rock,561,43.154
USA,4,Alternative & Punk,130,10.0
USA,3,Metal,124,9.538
USA,14,R&B/Soul,53,4.077
USA,6,Blues,36,2.769
USA,23,Alternative,35,2.692
USA,7,Latin,22,1.692
USA,9,Pop,22,1.692
USA,17,Hip Hop/Rap,20,1.538
USA,2,Jazz,14,1.077


### The Solution

Above, one can see which genres are the most popular in the USA. Rock is the most popular genre by some distance.

Based on the popularity of certain genres above, of the four artists listed earlier I should recommend the purchase of albums from:
- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)

The album from Regal (Hip-Hop) would not be purchased from the list of four artists on the basis of genre popularity.

## Analyzing Sales Support Agent Performance

### The Problem

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. I will analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

It's important to consider whether any extra columns from the employee table will explain any variance I will see or whether the variance might instead be indicative of employee performance.

**I will write a query that finds the total dollar amount of sales assigned to each sales support agent within the company.** I'll add any extra attributes for that employee that I find are relevant.

In [4]:
%%sql

SELECT
    c.support_rep_id AS employee_number,
    e.first_name || ' ' || e.last_name AS employee_name,
    SUM(i.total) AS total_sales,
    e.hire_date AS date_hired,
    e.birthdate AS birthday,
    e.title AS job_title,
    e.reports_to AS supervised_by,
    e.postal_code AS postal_code
FROM customer AS c
    LEFT JOIN employee AS e
        ON c.support_rep_id = e.employee_id
    LEFT JOIN invoice AS i
        ON i.customer_id = c.customer_id
GROUP BY employee_number
ORDER BY total_sales DESC

Done.


employee_number,employee_name,total_sales,date_hired,birthday,job_title,supervised_by,postal_code
3,Jane Peacock,1731.510000000004,2017-04-01 00:00:00,1973-08-29 00:00:00,Sales Support Agent,2,T2P 5M5
4,Margaret Park,1584.0000000000032,2017-05-03 00:00:00,1947-09-19 00:00:00,Sales Support Agent,2,T2P 5G3
5,Steve Johnson,1393.9200000000028,2017-10-17 00:00:00,1965-03-03 00:00:00,Sales Support Agent,2,T3B 1Y7


### The Solution

At a glance, the amount of sales per each of the sales support agents seems to be based on the date hired. Other factors do not seem to impact the 'total_sales' column nearly as strongly.

## Customer Sales by Country

### The Problem

My next task is to analyze the sales data for customers from each different country. I have been given guidance to use the country value from the customers table and ignore the country from the billing address in the invoice table.

**In particular, I have been directed to calculate data, for each country, on the:**

- **total number of customers**
- **total value of sales**
- **average value of sales per customer**
- **average order value**

In [5]:
%%sql

WITH country_vs_other AS
    (SELECT
        CASE
            WHEN (SELECT count(*) FROM customer WHERE country = c.country) = 1 
                THEN 'Other'
            ELSE c.country
            END AS country,
        c.customer_id,
        il.*
    FROM invoice_line AS il
        INNER JOIN invoice i ON i.invoice_id = il.invoice_id
        INNER JOIN customer c ON c.customer_id = i.customer_id)
    
SELECT
    country,
    COUNT(DISTINCT customer_id) AS total_customers,
    SUM(unit_price) AS total_sales,
    SUM(unit_price) / COUNT(DISTINCT customer_id) AS average_value_per_customer,
    SUM(unit_price) / COUNT(DISTINCT invoice_id) AS average_order_value,
    sort
FROM
    (
    SELECT
        cvo.*,
        CASE
            WHEN cvo.country = 'Other'
                THEN 0
                ELSE 1
            END AS sort
    FROM
    country_vs_other AS cvo        
    )
GROUP BY country
ORDER BY sort DESC, total_sales DESC

Done.


country,total_customers,total_sales,average_value_per_customer,average_order_value,sort
USA,13,1040.490000000008,80.03769230769292,7.942671755725252,1
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309,1
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647,1
France,5,389.0700000000021,77.81400000000042,7.781400000000042,1
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186,1
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034,1
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457,1
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835,1
India,2,183.1500000000002,91.5750000000001,8.72142857142858,1
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486,0


### The Solution

Above I've generated the number of total customers by country, their total sales, the average value per customer by country, and the average order value of customers by country.

## Informing the Music Purchasing Strategy

### The Problem

The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

- purchase a whole album
- purchase a collection of one or more individual tracks

The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management is considering changing the purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies instead of purchasing every track from an album.

I want to find out what percentage of purchases are individual track purchases versus whole album purchases so that management can use this data to understand the effect this decision might have on overall revenue.

**I'll write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:**

- **Number of invoices**
- **Percentage of invoices**

I'm going to have to identify whether each invoice has all the tracks from an album. I can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. 

I can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track I pick, since if it's an album purchase, that album will be the same for all tracks.

In [6]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


### The Solution

Only 18.6% of purchases are entire album purchases. While more information would be helpful to make a recommendation on purchasing strategy, changing the purchase strategy *may* save money for the store.

However, changing the purchasing strategy is a major change and one that may negatively impact the purchasing experience of some customers. More data is required, perhaps a customer survey, before a recommendation can be made. 